In [1]:
#!/usr/bin/env python3

!pip install requests
"""
Utility script to programmatically submit multiple random survey responses.

This script talks directly to the multi‑step Flask survey using simple HTTP
requests. It maintains a session (via cookies) for each survey submission,
fetches the CSRF token for every step, and posts the appropriate fields.

Usage:
    python submit_random_responses.py

Before running, set the BASE_URL variable below to the root URL of your
deployed survey, for example "http://localhost:8000" or
"https://your-survey.example.com". The script will attempt to send ten
independent submissions. Feel free to adjust `NUM_SUBMISSIONS` if you want
more or fewer responses.

Requirements:
    This script depends only on the standard library and the third‑party
    `requests` package. If `requests` is not installed, you can install it
    via pip: `pip install requests`. No browser automation or Selenium is
    required, making it light‑weight and easy to run.

Important:
    The script assumes the survey flow and field names match those defined
    in the provided Flask code (step1..step4). If you customize your
    templates or change field names, adjust the payload construction
    accordingly.
"""

import random
import re
import string
import sys
import time
from typing import Dict, Tuple

try:
    import requests
except ImportError as exc:
    sys.exit(
        "The 'requests' library is required to run this script.\n"
        "Install it with: pip install requests"
    )

# ===== Configuration =====
# Base URL of your survey. Include protocol and no trailing slash.
# For example: "http://localhost:8000" or "https://my-survey.app".
# The script reads this variable when building request URLs.
BASE_URL = "https://survey-testing.onrender.com"

# Number of random submissions to create
NUM_SUBMISSIONS = 100

# Delay (in seconds) between each submission. A small delay prevents
# hammering your server. Set to 0 for no delay.
DELAY_BETWEEN = 0.5


def get_csrf_token(html: str) -> str:
    """Extract the CSRF token from the HTML form.

    The token is expected in a hidden input field like:
        <input type="hidden" name="_csrf" value="TOKEN">

    Args:
        html: The raw HTML content as a string.
    Returns:
        The CSRF token string.
    Raises:
        ValueError: If the token cannot be found in the page.
    """
    match = re.search(r'name="_csrf"\s+value="([^"]+)"', html)
    if not match:
        raise ValueError("Unable to find CSRF token in the page")
    return match.group(1)


def random_name() -> str:
    """Generate a random Vietnamese‑style name for the respondent.

    This helper picks from a selection of common first names and randomly
    generates a surname using letters. While the names aren't meant to
    represent real individuals, they produce plausible entries.
    """
    first_names = [
        "An", "Bình", "Chi", "Duy", "Hà", "Huy", "Lan", "Minh", "Ngọc",
        "Phúc", "Quân", "Thảo", "Trang", "Vy"
    ]
    surname = ''.join(random.choices(string.ascii_uppercase, k=1)) + ''.join(
        random.choices(string.ascii_lowercase, k=random.randint(4, 7))
    )
    return f"{surname} {random.choice(first_names)}"


def random_feedback() -> str:
    """Generate a short random feedback sentence or return an empty string.

    Feedback is optional in the survey. To simulate realistic behaviour,
    roughly half of the submissions will include a brief comment. The
    sentences are drawn from a small pool of generic phrases.
    """
    comments = [
        "Dịch vụ khá tốt, tôi hài lòng.",
        "Cần cải thiện tốc độ giao hàng.",
        "Mong có nhiều khuyến mãi hơn.",
        "Sản phẩm chất lượng, sẽ mua tiếp.",
        "Giá hơi cao so với thị trường.",
    ]
    return random.choice(comments) if random.random() < 0.5 else ""


def submit_survey(session: requests.Session) -> Tuple[bool, str]:
    """Perform one complete survey submission with random data.

    This function carries out the four steps of the survey, handling
    CSRF tokens and preserving session state. It reports success or
    failure and returns a message summarising the outcome.

    Args:
        session: An existing requests.Session object to persist cookies.
    Returns:
        (success, message): A tuple where success indicates whether the
            submission finished without errors, and message gives a
            human‑readable explanation.
    """
    try:
        # Step 1 – fetch form and submit demographics
        r = session.get(f"{BASE_URL}/step1")
        r.raise_for_status()
        csrf_token = get_csrf_token(r.text)
        # Build random answers for step1
        name = random_name()
        role = random.choice(["Owner", "Manager", "Staff"])
        store_type = random.choice([
            "Grocery", "Pharmacy", "Baby Store", "Other"
        ])
        payload1 = {
            "_csrf": csrf_token,
            "name": name,
            "role": role,
            "store_type": store_type,
        }
        r = session.post(
            f"{BASE_URL}/step1", data=payload1, allow_redirects=True
        )
        r.raise_for_status()

        # Step 2 – satisfaction and frequency
        r = session.get(f"{BASE_URL}/step2")
        r.raise_for_status()
        csrf_token = get_csrf_token(r.text)
        satisfaction = random.randint(1, 5)
        frequency = random.choice(["Weekly", "Monthly", "Less often"])
        payload2 = {
            "_csrf": csrf_token,
            "satisfaction": str(satisfaction),
            "frequency": frequency,
        }
        r = session.post(
            f"{BASE_URL}/step2", data=payload2, allow_redirects=True
        )
        r.raise_for_status()

        # Step 3 – brand perception and optional diaper brand
        r = session.get(f"{BASE_URL}/step3")
        r.raise_for_status()
        csrf_token = get_csrf_token(r.text)
        brand_perception = random.choice([
            "Agree", "Neutral", "Disagree"
        ])
        # Determine diaper brand only for baby stores
        primary_diaper_brand = ""
        if store_type == "Baby Store":
            # Choose from known diaper brands or random text
            diaper_brands = [
                "Huggies", "Pampers", "Bobby", "Moony", "Goon"
            ]
            primary_diaper_brand = random.choice(diaper_brands)
        payload3 = {
            "_csrf": csrf_token,
            "brand_perception": brand_perception,
            "primary_diaper_brand": primary_diaper_brand,
        }
        r = session.post(
            f"{BASE_URL}/step3", data=payload3, allow_redirects=True
        )
        r.raise_for_status()

        # Step 4 – open feedback (optional)
        r = session.get(f"{BASE_URL}/step4")
        r.raise_for_status()
        csrf_token = get_csrf_token(r.text)
        open_feedback = random_feedback()
        payload4 = {
            "_csrf": csrf_token,
            "open_feedback": open_feedback,
        }
        r = session.post(
            f"{BASE_URL}/step4", data=payload4, allow_redirects=True
        )
        r.raise_for_status()

        # Final page should be /thanks. Check to confirm success.
        if r.url.endswith("/thanks"):
            return True, f"Submitted: {name} ({role}, {store_type})"
        else:
            return False, f"Unexpected redirect to {r.url}"

    except Exception as exc:
        return False, f"Error during submission: {exc}"


def main() -> None:
    print(f"Sending {NUM_SUBMISSIONS} survey submissions to {BASE_URL}...")
    successes = 0
    for i in range(NUM_SUBMISSIONS):
        session = requests.Session()
        ok, message = submit_survey(session)
        prefix = f"[{i+1}/{NUM_SUBMISSIONS}]"
        if ok:
            successes += 1
            print(prefix, "✓", message)
        else:
            print(prefix, "✗", message)
        if i < NUM_SUBMISSIONS - 1 and DELAY_BETWEEN > 0:
            time.sleep(DELAY_BETWEEN)
    print(
        f"Finished. Successful submissions: {successes}/{NUM_SUBMISSIONS}"
    )


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("Aborted by user.")


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Sending 100 survey submissions to https://survey-testing.onrender.com...
[1/100] ✓ Submitted: Yzkimoai Vy (Manager, Baby Store)
[2/100] ✓ Submitted: Jyeycn Ngọc (Owner, Other)
[3/100] ✓ Submitted: Rlktpiqd Lan (Owner, Grocery)
[4/100] ✓ Submitted: Hmvar Chi (Staff, Pharmacy)
[5/100] ✓ Submitted: Uoctxdoo An (Manager, Other)
[6/100] ✓ Submitted: Odmvng Minh (Owner, Other)
[7/100] ✓ Submitted: Kvuezr Lan (Owner, Pharmacy)
[8/100] ✓ Submitted: Puumj Vy (Staff, Pharmacy)
[9/100] ✓ Submitted: Puuctt Duy (Owner, Pharmacy)
[10/100] ✓ Submitted: Dcgvfabn Ngọc (Manager, Grocery)
[11/100] ✓ Submitted: Sedxgmeh Trang (Manager, Baby Store)
[12/100] ✓ Submitted: Tqsgzssj Ngọc (Owner, Grocery)
[13/100] ✓ Submitted: Jgcearel Duy (Manager, Baby Store)
[14/100] ✓ Submitted: Fjyeto Minh (Owner, Grocery)
[15/100] ✓ Submitted: Sdnpv Chi (Manager, Other)
[16/100] ✓ Submitted: Naehzu Minh (Manager,